This notebook was inpired by [this LlamaIndex notebook](https://colab.research.google.com/drive/1GyPRMiwxS7rKxKpRt4r-ckYfmAw2GxdQ?usp=sharing)

Making some changes to it with the only intention of trying ideas and learning.

Notice that I am assuming you have the relevant API_KEYs as environmental variables.

In [10]:
from bubls.utils.data.download import download_file_from_url
from bubls.utils.indexing import create_index_from_path
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import QueryEngineTool, ToolMetadata
import os

## Defining global variables

In [2]:
METADATA = {
    "uber_march_2022": {
        "source_url": "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf",
        "file_name": "uber_march_2022.pdf",
        "save_data_to": os.path.join(os.environ["DATA_DIR"], "uber_q"),
        "persist_index_to": os.path.join(os.environ["PERSIST_DIR"], "uber_q"),
        },
    "uber_june_2022": {
        "source_url": "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf",
        "file_name": "uber_june_2022.pdf",
        "save_data_to": os.path.join(os.environ["DATA_DIR"], "uber_q"),
        "persist_index_to": os.path.join(os.environ["PERSIST_DIR"], "uber_q"),
        },
    "uber_sept_2022": {
        "source_url": "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_sept_2022.pdf",
        "file_name": "uber_sept_2022.pdf",
        "save_data_to": os.path.join(os.environ["DATA_DIR"], "uber_q"),
        "persist_index_to": os.path.join(os.environ["PERSIST_DIR"], "uber_q"),
        },
}

## Ingest Data
- Download Information
- Create&Persist or Load Index 
- Create Query Engine

In [3]:
query_engine_dict = {}
for k, md in METADATA.items():
    download_file_from_url(md["source_url"], md["file_name"], md["save_data_to"])
    index = create_index_from_path(
        md["persist_index_to"],
        md["save_data_to"],
        # {".pdf": parser}
    )
    query_engine_dict[k] = index.as_query_engine(similarity_top_k=3)

Loading Index


Loading Index
Loading Index


## Define tool

In [4]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine_dict[k],
        metadata=ToolMetadata(
            name=k,
            description=(
                f"Provides information about {k} financials. "
            ),
        ),
    )
    for k in METADATA.keys()
]

## Agent stepwise

In [5]:
llm_gpt4 = OpenAI(model="gpt-4")
gpt4_agent_worker = FunctionCallingAgentWorker.from_tools(query_engine_tools, llm=llm_gpt4, verbose=True, allow_parallel_tool_calls=False)
gpt4_agent = AgentRunner(gpt4_agent_worker)

In [6]:
task = gpt4_agent.create_task("Analyze the changes in R&D expenditures and revenue for Uber in June 2022 or March 2022")
completed = False
while not completed:
    step_output = gpt4_agent.run_step(task.task_id)
    completed = step_output.is_last
    if completed:
        print("Completed task")
    else:
        print("Task not completed, moving to next task.")


Added user message to memory: Analyze the changes in R&D expenditures and revenue for Uber in June 2022 or March 2022


=== Calling Function ===
Calling function: uber_march_2022 with args: {"input": "What were the R&D expenditures and revenue for Uber in March 2022?"}
=== Function Output ===
The Research and Development expenditures for Uber in March 2022 were $587 million. The revenue for Uber in March 2022 was $6,854 million.
Task not completed, moving to next task.
=== Calling Function ===
Calling function: uber_june_2022 with args: {"input": "What were the R&D expenditures and revenue for Uber in June 2022?"}
=== Function Output ===
The Research and Development expenditures for Uber in June 2022 were $587 million. The revenue for Uber in June 2022 was $6,854 million.
Task not completed, moving to next task.
=== LLM Response ===
The Research and Development (R&D) expenditures for Uber remained constant at $587 million in both March 2022 and June 2022. Similarly, the revenue for Uber also remained the same at $6,854 million in both these months.
Completed task


## Adding Human Feedback

In [7]:
task = gpt4_agent.create_task("Analyze the changes in R&D expenditures and revenue from march to june")

In [8]:
step_output = gpt4_agent.run_step(task.task_id)

Added user message to memory: Analyze the changes in R&D expenditures and revenue from march to june


=== Calling Function ===
Calling function: uber_march_2022 with args: {"input": "What were the R&D expenditures and revenue in March 2022?"}
=== Function Output ===
The Research and Development expenditures in March 2022 were $587 million, and the revenue in March 2022 was $6,854 million.


In [9]:
step_output = gpt4_agent.run_step(task.task_id, input='What about September?')

Added user message to memory: What about September?
=== Calling Function ===
Calling function: uber_sept_2022 with args: {"input": "What were the R&D expenditures and revenue in September 2022?"}
=== Function Output ===
The Research and Development expenditures were $587 million in March 2022. The revenue in March 2022 was $6,854 million.
